In [1]:
using Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("BenchmarkTools")
using BenchmarkTools
using DataFrames
using CSV
using Base.Threads

In [2]:
# function bufferRatings(filename)
#     rows = 250000
#     res = DataFrame()
#     buffer = DataFrame()
#     movieso = DataFrame(CSV.File("movies.csv"))
#     for row in CSV.Rows("ratings.csv")
#         push!(buffer, row)
#         println(nrow(buffer))
#         if nrow(buffer) > rows
#             println("processing chunk")
#             res = [res;countGenres(buffer,movieso)]
#             buffer = DataFrame()
#         end
#     res = [res;countGenres(buffer,movieso)]
#     end
# end

In [5]:
function readStuff(filename)
    if lowercase(reverse(reverse(filename)[1:4])) == ".csv"
        ratings = DataFrame(CSV.File(filename,header=false))
        ratings = rename(ratings, :Column1 => :userId, :Column2 => :movieId)
    else
        println("pasame un csv plis")
        return 
    end
    return ratings
end

readStuff (generic function with 1 method)

In [2]:
function readStuffOld(filename)
    if lowercase(reverse(reverse(filename)[1:4])) == ".csv"
        ratings = DataFrame(CSV.File(filename))
    else
        println("pasame un csv plis")
        return 
    end
    return ratings
end

readStuffOld (generic function with 1 method)

In [3]:
function countGenres(ratingsIn,moviesIn)
    ratings = select(ratingsIn,:movieId)
    movies = select(
                    flatten(
                        transform!(moviesIn, :genres => ByRow(x -> ismissing(x) ? [missing] : string.(split(x, "|"))) => :flattened_genres)
                        ,:flattened_genres
                        )
                    ,:movieId,:flattened_genres =>:genres
                    )
    mergedf = innerjoin(movies,ratings, on = :movieId)
    sol = sort(combine(groupby(mergedf,:genres), nrow => :count),:genres)
    return sol
end

countGenres (generic function with 1 method)

In [ ]:
function bufferRatings()
    res = [DataFrame() for _ in 1:10]
    movieso = DataFrame(CSV.File("movies.csv"))
    @threads for i in [0,1,2,3,4,5,6,7,8,9]
        chunkFilename = string("ratings",i,".csv")
        res[i+1] = countGenres(readStuff(chunkFilename),movieso)
    end
    finres = DataFrame()
    for i in [1,2,3,4,5,6,7,8,9]
        finres = [finres;res[i]]
    end
    finres = sort(combine(groupby(finres,[:genres]), :count => sum => :count),:genres)
    return finres
end

bufferRatings (generic function with 1 method)

In [4]:
@btime res = countGenres(readStuffOld("ratings.csv"),readStuffOld("movies.csv"))

  6.488 s (799024 allocations: 6.38 GiB)


Row,genres,count
,String,Int64
1,(no genres listed),26627
2,Action,7446918
3,Adventure,5832424
4,Animation,1630987
5,Children,2124258
6,Comedy,8926230
7,Crime,4190259
8,Documentary,322449
9,Drama,10962833


In [ ]:
@btime a = bufferRatings()

  6.831 s (6861713 allocations: 7.56 GiB)


Row,genres,count
,String,Int64
1,(no genres listed),21606
2,Action,5954382
3,Adventure,4660324
4,Animation,1304480
5,Children,1697255
6,Comedy,7140669
7,Crime,3351576
8,Documentary,260968
9,Drama,8768664
